# Pipeline SDK

In [1]:
import kfp

@kfp.dsl.pipeline(name='my-pipeline_1207-0027', description='Hello How do you do')
def hello_pipeline():
    train_task = hello_component()

@kfp.dsl.component
def hello_component():
    return kfp.dsl.ContainerOp(
        name='my-ContainerOp',
        image='kangwoo/kfp-mnist:kfp' # TODO 본 교육용으로 바꿀 것
    )    

# Pipeline 컴파일

In [2]:
kfp.compiler.Compiler().compile(hello_pipeline, 'my_pipeline.zip')

In [3]:
! ls -l my_pipeline.zip

-rw-r--r-- 1 jovyan users 460 Dec  6 15:37 my_pipeline.zip


In [5]:
! unzip -l my_pipeline.zip

Archive:  my_pipeline.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
      804  1980-01-01 00:00   pipeline.yaml
---------                     -------
      804                     1 file


In [6]:
! unzip -p my_pipeline.zip

"apiVersion": |-
  argoproj.io/v1alpha1
"kind": |-
  Workflow
"metadata":
  "annotations":
    "pipelines.kubeflow.org/pipeline_spec": |-
      {"description": "Hello How do you do", "name": "my-pipeline_1207-0027"}
  "generateName": |-
    my-pipeline-1207-0027-
"spec":
  "arguments":
    "parameters": []
  "entrypoint": |-
    my-pipeline-1207-0027
  "serviceAccountName": |-
    pipeline-runner
  "templates":
  - "container":
      "image": |-
        kangwoo/kfp-mnist:kfp
    "metadata":
      "annotations":
        "pipelines.kubeflow.org/component_spec": |-
          {"name": "Hello component"}
    "name": |-
      my-containerop
  - "dag":
      "tasks":
      - "name": |-
          my-containerop
        "template": |-
          my-containerop
    "name": |-
      my-pipeline-1207-0027


# Pipeline Experiment

In [7]:
client = kfp.Client()

my_experiment = client.create_experiment(
    name='My first Experiment' # Experiment name
)

# Pipeline 실행

In [8]:
my_run = client.run_pipeline(
    my_experiment.id, 
    'oh_my_experiment', # Run name
    'my_pipeline.zip'
)